The aim of this workbook is to process a set of active faults, calculate the recurrence from slip and then build an OpenQuake source model

In [ ]:
 # -*- coding: utf-8 -*-
%matplotlib inline

# General Python imports
from __future__ import unicode_literals
import numpy as np
import matplotlib.pyplot as plt

# OpenQuake imports
from openquake.hazardlib.geo import Point, Polygon, Line, SimpleFaultSurface
from openquake.hazardlib.scalerel import WC1994
from openquake.hazardlib.source import SimpleFaultSource
from openquake.hazardlib.tom import PoissonTOM
from openquake.hazardlib.sourcewriter import write_source_model

# Some additional utilities to help this process
import utilities as utils

# Input Files
fault_file = "./data/East_Alpine_Faults.shp"
area_file = "./data/East_Alpine_Areas.shp"


Here are some methods that illustrate how to calculate basic fault geometry properties

In [ ]:
def get_dimensions(fault):
    """
    Get the dimensions (length, width and area) of the fault
    taking into account the upper and lower dip estimates
    """
    # Calculate Length
    length = fault["trace"].get_length()
    # Get seismogenic thickness of fault
    d_z = fault["LSD"] - fault["USD"]
    # Calculate width
    width_l = d_z / np.sin(np.radians(fault["dip"][1]))
    width_u = d_z / np.sin(np.radians(fault["dip"][0]))
    # Calculate area
    area_l = length * width_l
    area_u = length * width_u
    return length, width_l, width_u, area_l, area_u

def get_moment(fault, mu=3.0E10):
    """
    Calculate the moment rate and areas of the fault for all the
    possible combinations of dip and area
    """
    # Get the dimensions
    length, width_l, width_u, area_l, area_u = get_dimensions(fault)
    # Produces 4 possible moment estimates:
    # 1 - low area, low slip
    mo_1 = mu * (area_l * 1.0E6) * (fault["slip"][0] * 1.0E-3)
    # 2 - low area, high slip
    mo_2 = mu * (area_l * 1.0E6) * (fault["slip"][1] * 1.0E-3)
    # 3 - high area, low slip
    mo_3 = mu * (area_u * 1.0E6) * (fault["slip"][0] * 1.0E-3)
    # 4 - high area, high slip
    mo_4 = mu * (area_u * 1.0E6) * (fault["slip"][1] * 1.0E-3)
    return [mo_1, mo_2, mo_3, mo_4], [area_l, area_l, area_u, area_u]

def get_mchar(fault, msr=WC1994()):
    """
    Get the characteristic (or Mmax) magnitude given the
    area (and dip options)
    """
    # Get the two area estimates
    _, _, _, area_l, area_u = get_dimensions(fault)
    # Get the average rake
    rake_l, rake_u = fault["rake"]
    rake = ((rake_l + rake_u) / 2.)
    if rake > 180:
        # Check if rake is outside the -180 - 180 range
        rake -= 360.0
    return msr.get_median_mag(area_l, rake),\
        msr.get_median_mag(area_u, rake)

def get_mean_dimensions(fault, msr=WC1994()):
    """
    Returns dimensions of the mean fault surface
    """
    # Arithmetic mean fault dip
    mean_dip = (fault["dip"][0] + fault["dip"][1]) / 2.0
    # Corresponding length, width and area
    length = fault["trace"].get_length()
    d_z = fault["LSD"] - fault["USD"]
    width = d_z / np.sin(np.radians(mean_dip))
    area = length * width
    # Get the average rake
    rake_l, rake_u = fault["rake"]
    rake = ((rake_l + rake_u) / 2.)
    if rake > 180:
        rake -= 360.0
    # Get Mmax
    mmax = msr.get_median_mag(area, rake)
    # Build surface
    surface = SimpleFaultSurface.from_fault_data(fault["trace"],
                                                 fault["USD"],
                                                 fault["LSD"],
                                                 mean_dip,
                                                 mesh_spacing=1.0)
    return area, rake, mean_dip, mmax, surface


def plot_activity_rates(mags, cum_rates, incr_rates, xlims=(0, 9), title=""):
    """
    Create a bar plot showing the incremental and cumulative rates
    """
    plt.figure(figsize=(12,9))
    plt.bar(mags, cum_rates, 0.1, facecolor="r", edgecolor="w",log=True, label="Cumulative")
    plt.bar(mags, incr_rates, 0.1, facecolor="b", edgecolor="b",log=True, alpha=0.5, label="Incremental")
    plt.xlabel("Mw", fontsize=16)
    plt.ylabel("Rate", fontsize=16)
    plt.xlim(xlims[0], xlims[1])
    plt.grid()
    plt.legend(fontsize=16)
    if title:
        plt.title(title, fontsize=18)

### Load in the Fault Data

In [ ]:
# Load in the faults from a shapefile (use a utility function for this)
faults = utils.fault_shapefile_to_dictionary(fault_file)

for i, flt in enumerate(faults):
    print "==========================================="
    print "===      %s     ====" % str(i)
    print "==========================================="
    for key in flt:
        print key, flt[key]
   

### Calculating & Viewing Different Recurrence Models

##### Exponential

In [ ]:
# Get the basic dimensions
_, _, _, area_u, area_l = get_dimensions(faults[4])
mmax_l, mmax_u = get_mchar(faults[4])
# Setup the confiration
config = {
    "model": "Exponential",
    "MFD_spacing": 0.1,
    "Minimum_Magnitude": 4.5,
    "b_value": [1.0, 0.1]
}

# Calculate the activity rates
mags, incr_rates, cum_rates, mf_dist = utils.build_fault_model(area_u, faults[4]["slip"][1], mmax_u, config)
# Build the plot
plot_activity_rates(mags, cum_rates, incr_rates, xlims=(4.5, 8.0))


##### Hybrid

In [ ]:
_, _, _, area_u, area_l = get_dimensions(faults[4])
mmax_l, mmax_u = get_mchar(faults[4])
config = {
    "model": "Hybrid",
    "MFD_spacing": 0.1,
    "Minimum_Magnitude": 4.5,
    "b_value": [1.0, 0.1]
}
mags, incr_rates, cum_rates, mf_dist = utils.build_fault_model(area_u, faults[4]["slip"][1], mmax_u, config)
plot_activity_rates(mags, cum_rates, incr_rates, xlims=(4.5, 8.0))

##### Characteristic 

In [ ]:
_, _, _, area_u, area_l = get_dimensions(faults[4])
mmax_l, mmax_u = get_mchar(faults[4])
config = {
    "model": "Characteristic",
    "MFD_spacing": 0.1,
    "Sigma": 0.1,
    "Lower_Bound": -3.0,
    "Upper_Bound": 3.0
}

mags, incr_rates, cum_rates, mf_dist = utils.build_fault_model(area_u, faults[4]["slip"][1], mmax_u, config)

plot_activity_rates(mags, cum_rates, incr_rates, xlims=(4.5, 8.0))

### Build a Fault Source Model

In [ ]:
def build_OQ_sources(faults, config, msr=WC1994()):
    """
    Builds the OpenQuake Source Model - simple logic tree: mean dip, mean area, geometric mean slip
    """
    sources = []
    for fault in faults:
        # Get the mean dimensions
        area, rake, dip, mmax, surface = get_mean_dimensions(fault, msr)
        # Get geometric mean slip
        slip = np.sqrt(fault["slip"][0] * fault["slip"][1])
        # Get the mfd
        mags, rates, cumulative_rates, fault_mfd = utils.build_fault_model(area, slip, mmax, config)
        # Plot the rates
        plot_activity_rates(mags, cumulative_rates, rates, xlims=(4.5, 8.0), title=fault["ID"])
        # Build the OpenQuake Source
        source = SimpleFaultSource(fault["ID"], fault["name"], "Active Shallow Crust",
                                   fault_mfd, 1.0, msr, 1.0, PoissonTOM(1.0), fault["USD"],
                                   fault["LSD"], fault["trace"], dip, rake)
        sources.append(source)
    return sources
        

Build the faults assuming a hybrid model on all

In [ ]:
config = {
    "model": "Hybrid",
    "MFD_spacing": 0.1,
    "Minimum_Magnitude": 4.5,
    "b_value": [1.0, 0.1]
}
sources = build_OQ_sources(faults, config, msr=WC1994())

Export the source model to an xml file

In [ ]:
write_source_model("East_Alpine_Faults.xml", sources, name="East Alpine Fault Source Model")